In [12]:
#Multi Layer Perceptron for Binary Classification
import tensorflow as tf
from matplotlib import pyplot
import numpy as np
import pandas as pd
from pandas import read_csv
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [13]:
#Load the Dataset
path = 'raw_mosaic_pixeldata_NN.csv'
df = read_csv(path, header=None)
print(df)

            0         1         2         3         4         5         6    \
0      0.015129  0.017095  0.019158  0.020151  0.020051  0.020037  0.020417   
1      0.021492  0.021839  0.022099  0.021414  0.019458  0.017557  0.017055   
2      0.018931  0.018543  0.018299  0.017018  0.014979  0.014023  0.015348   
3      0.017672  0.017455  0.017240  0.016048  0.014647  0.014277  0.015218   
4      0.019671  0.020411  0.020497  0.020230  0.021146  0.022208  0.022273   
...         ...       ...       ...       ...       ...       ...       ...   
36063  0.020789  0.019660  0.020122  0.021130  0.020752  0.018410  0.015181   
36064  0.036120  0.034278  0.034476  0.035099  0.034775  0.034062  0.033057   
36065  0.040596  0.039176  0.036893  0.034860  0.033695  0.032679  0.031467   
36066  0.037533  0.036553  0.034959  0.032973  0.031371  0.030838  0.031487   
36067  0.045737  0.040986  0.035687  0.031960  0.030897  0.031809  0.033327   

            7         8         9    ...       758 

In [14]:
#Load the Feature Selected Bands
feature_path = 'ETC_RFE_ANOVA.csv'
feature_df = read_csv(feature_path, header=None)
feature_array_triple = feature_df.to_numpy()
print(feature_array_triple)

feature_path = '12_features.csv'
feature_df = read_csv(feature_path, header=None)
feature_array_twelve = feature_df.to_numpy()
print(feature_array_twelve)

def array_values(list):
    for i in list:
        return i

[[ 24  25  26  27  28  29  30  31  32  33  34  35  78  79  81  82  84  86
   87  88  89  91  92  93 155 156 157 158 159 160 161 162 163 164 165 166
  167 168 169 170 171 172 173 186 187 188 189 191 192 193 194 196 197 198
  199 200 201 203 204 205 206 207 208 210 281 283 284 290 292 293 306 309
  313 319 321 327 330 331 336 337 338 619 620 621 622 623 624 710 714 718
  719 721 727 732 735 738 743 744 747 749 751 758 759 766]]
[[164 165 166 189 191 192 193 196 199 200 203 204]]


In [15]:
#Split into input/output columns
#:-1
#array_values()-1
X, y = df.values[:, array_values(feature_array_triple)-1], df.values[:, -1]
print(X)
print(y)

[[0.024323406 0.024786333 0.025557835 ... 0.07704927 0.078165427
  0.074911892]
 [0.017354405 0.017653821 0.018485628 ... 0.050072946 0.051070999
  0.047248427]
 [0.014408262 0.014687239 0.015662696 ... 0.051737711 0.052510031
  0.051437993]
 ...
 [0.032839824 0.035231948 0.037352055 ... -0.006028205 -0.005399798
  -0.023646984]
 [0.034997389 0.037384901 0.040153164 ... 0.047700714 0.048524681
  0.031022113]
 [0.041136507 0.043989088 0.046262812 ... 0.071768932 0.072871484
  0.059392739]]
['green' 'green' 'green' ... 'blue' 'blue' 'blue']


In [16]:
#Ensure Data are Floats
X = X.astype('float32')

In [17]:
#Encode Strings to integers
y = LabelEncoder().fit_transform(y)
print(y)

[1 1 1 ... 0 0 0]


In [18]:
#10 Fold Cross Validation
seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, y):
    #Determine number of input features
    n_features = X[train].shape[1]
    #Define Model
    N = 100
    model = Sequential()
    model.add(Dense(N, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
    model.add(Dense(2*N, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(3*N, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(2*N, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(1, activation='sigmoid'))
    #Compile Model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    #Fit Model
    history = model.fit(X[train], y[train], epochs=25, batch_size=10)
    scores = model.evaluate(X[test], y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/25
3247/3247 [==============================] - 4s 1ms/step - loss: 0.4154 - accuracy: 0.7988
Epoch 2/25
3247/3247 [==============================] - 4s 1ms/step - loss: 0.2762 - accuracy: 0.8742
Epoch 3/25
3247/3247 [==============================] - 4s 1ms/step - loss: 0.2252 - accuracy: 0.9039
Epoch 4/25
3247/3247 [==============================] - 4s 1ms/step - loss: 0.1921 - accuracy: 0.9174
Epoch 5/25
3247/3247 [==============================] - 4s 1ms/step - loss: 0.1739 - accuracy: 0.9267
Epoch 6/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1703 - accuracy: 0.9269
Epoch 7/25
3247/3247 [==============================] - 4s 1ms/step - loss: 0.1639 - accuracy: 0.9322
Epoch 8/25
3247/3247 [==============================] - 4s 1ms/step - loss: 0.1540 - accuracy: 0.9340
Epoch 9/25
3247/3247 [==============================] - 4s 1ms/step - loss: 0.1465 - accuracy: 0.9376
Epoch 10/25
3247/3247 [==============================] - 4s 1ms/step - loss: 0.142

3247/3247 [==============================] - 5s 1ms/step - loss: 0.1776 - accuracy: 0.9242
Epoch 6/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1654 - accuracy: 0.9319
Epoch 7/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1540 - accuracy: 0.9348
Epoch 8/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1483 - accuracy: 0.9369
Epoch 9/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1410 - accuracy: 0.9410
Epoch 10/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1314 - accuracy: 0.9462
Epoch 11/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1343 - accuracy: 0.9466
Epoch 12/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1325 - accuracy: 0.9474
Epoch 13/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1276 - accuracy: 0.9478
Epoch 14/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1233 - acc

3247/3247 [==============================] - 4s 1ms/step - loss: 0.1412 - accuracy: 0.9415
Epoch 10/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1539 - accuracy: 0.9368
Epoch 11/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1347 - accuracy: 0.9431
Epoch 12/25
3247/3247 [==============================] - 4s 1ms/step - loss: 0.1304 - accuracy: 0.9462
Epoch 13/25
3247/3247 [==============================] - 4s 1ms/step - loss: 0.1298 - accuracy: 0.9479
Epoch 14/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1274 - accuracy: 0.9452
Epoch 15/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1218 - accuracy: 0.9501
Epoch 16/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1290 - accuracy: 0.9474
Epoch 17/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1153 - accuracy: 0.9532
Epoch 18/25
3247/3247 [==============================] - 4s 1ms/step - loss: 0.1143 -

3247/3247 [==============================] - 5s 1ms/step - loss: 0.1347 - accuracy: 0.9455
Epoch 14/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1321 - accuracy: 0.9441
Epoch 15/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1243 - accuracy: 0.9479
Epoch 16/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1252 - accuracy: 0.9480
Epoch 17/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1219 - accuracy: 0.9487
Epoch 18/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1211 - accuracy: 0.9505
Epoch 19/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1098 - accuracy: 0.9552
Epoch 20/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1039 - accuracy: 0.9580
Epoch 21/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1104 - accuracy: 0.9555
Epoch 22/25
3247/3247 [==============================] - 5s 1ms/step - loss: 0.1113 -

In [19]:
#Plot Learning Curves